<a href="https://colab.research.google.com/github/gaborchris/DeepReplace/blob/master/chars_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

try:
  from google.colab import files
except Exception:
  pass

import tensorflow as tf

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import pathlib

TensorFlow 2.x selected.


In [0]:
BATCH_SIZE = 64
NOISE_DIM = 100

num_to_char = {}
alphabet = 'abcdefghijklmnopqrstuvwxyz'
for i in range(0, 10):
    num_to_char[i] = str(i)
for i in range(0, 26):
    num_to_char[i+10] = alphabet[i].upper()
for i in range(0, 26):
    num_to_char[i+10+26] = alphabet[i].lower()

In [0]:
def make_generator_model(n_classes=10):
    # Create class embedding channel
    input_label = tf.keras.layers.Input(shape=(1,))
    label_embedding = tf.keras.layers.Embedding(n_classes, 50)(input_label)
    upscaling = tf.keras.layers.Dense(7*7*1)(label_embedding)
    upscaling = tf.keras.layers.Reshape((7, 7, 1))(upscaling)

    # create seed encoding network
    seed_input = tf.keras.layers.Input(shape=(NOISE_DIM,))
    seed_fc = tf.keras.layers.Dense(7*7*256, use_bias=False)(seed_input)
    seed_fc = tf.keras.layers.BatchNormalization()(seed_fc)
    seed_fc = tf.keras.layers.LeakyReLU()(seed_fc)
    seed_fc = tf.keras.layers.Reshape((7, 7, 256))(seed_fc)

    # merge embedding with seed encoder
    merge = tf.keras.layers.Concatenate()([seed_fc, upscaling])
    assert tuple(merge.shape) == (None, 7, 7, 256+1)

    x = layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)(merge)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    assert tuple(x.shape) == (None, 7, 7, 128)

    x = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    assert tuple(x.shape) == (None, 14, 14, 64)

    # TODO tanh function with output between [-1, 1]
    output = layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='sigmoid')(x)
    model = tf.keras.Model([seed_input, input_label], output)
    assert model.output_shape == (None, 28, 28, 3)

    return model

def make_discriminator_model(n_classes=10):
    # Create a class embedding
    input_label = tf.keras.layers.Input(shape=(1,))
    label_embedding = tf.keras.layers.Embedding(n_classes, 50)(input_label)
    upscaling = tf.keras.layers.Dense(28*28)(label_embedding)
    upscaling = tf.keras.layers.Reshape((28, 28, 1))(upscaling)

    # merge image with class embedding
    input_image = tf.keras.layers.Input(shape=(28, 28, 3))
    merge = tf.keras.layers.Concatenate()([input_image, upscaling])

    # define classification architecture
    x = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')(merge)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model([input_image, input_label], output)

    return model

In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


def generator_loss(predictions):
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    return cross_entropy(tf.ones_like(predictions), predictions)

In [0]:
def generate_and_save_images(model, epoch, test_input, test_labels):
    predictions = model([test_input, test_labels], training=False)

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, :])
        # TODO figure out how to upsample mean 0 and unit variance
        # plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.xticks([])
        plt.yticks([])
        plt.xlabel(num_to_char[test_labels[i][0]])

    plt.tight_layout()
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.close()
    try:
      files.download('image_at_epoch_{:04d}.png'.format(epoch))
    except Exception:
      pass

In [6]:
# Get ground truth data
data_dir = tf.keras.utils.get_file('English',
                                    origin='http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/EnglishImg.tgz',
                                    untar=True, extract=True)
data_dir = pathlib.Path(data_dir)
image_path = os.path.join(data_dir, "Img/GoodImg/Bmp")

# TODO mean 0 and unit variance
# image_generator = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=True,
#                                                                   featurewise_std_normalization=True)
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
image_ground_truth = image_generator.flow_from_directory(image_path, target_size=(28, 28),
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='sparse')


Found 7705 images belonging to 62 classes.


In [0]:
# Define Model
num_classes = 62

generator = make_generator_model(n_classes=num_classes)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator = make_discriminator_model(n_classes=num_classes)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                  discriminator_optimizer=discriminator_optimizer,
                                  generator=generator,
                                  discriminator=discriminator)


In [0]:
# Setup repeated predictions
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, NOISE_DIM])
seed_labels = np.random.randint(0, num_classes, num_examples_to_generate).reshape((-1, 1))
fakes = generator([seed, seed_labels], training=False)
generate_and_save_images(generator, 0, seed, seed_labels)

In [0]:
# Define training procedure
@tf.function
def train_step(images, ground_truth_labels):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])
    random_labels = np.random.randint(0, num_classes, BATCH_SIZE).reshape((-1, 1))

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fakes = generator([noise, random_labels], training=True)
        ground_truth_preds = discriminator([images, ground_truth_labels], training=True)
        fake_preds = discriminator([fakes, random_labels], training=True)

        gen_loss = generator_loss(fake_preds)
        disc_loss = discriminator_loss(real_output=ground_truth_preds, fake_output=fake_preds)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs, ckpt_prefix):
    print("Starting training")
    for epoch in range(epochs):
        start = time.time()
        i = 0
        for (image_batch, labels_batch) in dataset:
            train_step(image_batch, labels_batch)
            i += 1
            if i > len(dataset):
              break

        if (epoch + 1) % 5 == 0:
            generate_and_save_images(generator, epoch+1, seed, seed_labels)
            checkpoint.save(file_prefix=ckpt_prefix)
        print('Time for epoch {} is {} sec'.format(epoch+1, time.time()-start))


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
checkpoint_dir = '/content/drive/My Drive/char_gan_ckpts/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

In [0]:
status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
train(image_ground_truth, epochs=1000, ckpt_prefix=checkpoint_prefix)

Starting training
Time for epoch 1 is 9.334131717681885 sec
Time for epoch 2 is 5.750824451446533 sec
Time for epoch 3 is 5.749228239059448 sec
Time for epoch 4 is 5.806707859039307 sec
Time for epoch 5 is 7.739721059799194 sec
Time for epoch 6 is 5.892318248748779 sec
Time for epoch 7 is 5.872906923294067 sec
Time for epoch 8 is 5.870932579040527 sec
Time for epoch 9 is 5.875095844268799 sec
Time for epoch 10 is 7.317830324172974 sec
Time for epoch 11 is 5.771472930908203 sec
Time for epoch 12 is 5.825550079345703 sec
Time for epoch 13 is 5.834111928939819 sec
Time for epoch 14 is 5.737351894378662 sec
Time for epoch 15 is 7.27737832069397 sec
Time for epoch 16 is 5.729223728179932 sec
Time for epoch 17 is 5.789766550064087 sec
Time for epoch 18 is 5.803072214126587 sec
Time for epoch 19 is 5.764939785003662 sec
Time for epoch 20 is 7.77762508392334 sec
Time for epoch 21 is 5.813733339309692 sec
Time for epoch 22 is 5.834491968154907 sec
Time for epoch 23 is 5.699938535690308 sec
Time